In [1]:
import time
import numpy as np
import pylab as pl
import tensorflow as tf
import function as fn
%matplotlib inline
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = '' # AMD Ryzen Threadripper 2990WX 32-Core (CPU)
#os.environ["CUDA_VISIBLE_DEVICES"] = '0' # NVIDIA TITAN V (GPU)

os.environ["CUDA_VISIBLE_DEVICES"] = '1' # NVIDIA TITAN RTX (GPU)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session( config=config )

data = np.loadtxt('../vol.txt')[:,1:]
Maximum, Minimum = np.max(data[:,-1]), np.min(data[:,-1])
print(data.shape)

/home/kooeunho/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kooeunho/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kooeunho/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kooeunho/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

(3145, 16)


In [2]:
start_time = time.time()
ratio      = 0.7
hidden_dim = 5
lr1, lr2   = 0.005, 0.0005
batch_size = 512
epochs     = 200
us         = 2/10
howmany    = 50
depth      = 6
extreme_ratio = 0.2

results = []
predicted_vectors = []
for how in range(howmany):
    print(how)
    for factor in [1/10*i for i in range(1,10)]:
        trainX, trainY, testX, testY = fn.split_LSTM2(data, depth, ratio, factor)
        pred = fn.RNN(trainX, trainY, testX, testY, depth, hidden_dim, lr1, lr2, batch_size, epochs, us)
        pred[pred < 0] = 0
        pred = (pred**(1/factor))*(Maximum - Minimum) + Minimum
        predicted_vectors.append(pred)
        real  = data[:,-1][-len(pred):]
        rmse = fn.RMSE(real, pred)
        cc   = fn.CC(real, pred)
        area = fn.distribution(pred, real, 100)[-1]
        extreme = fn.extreme(pred, real, extreme_ratio)
        result = [factor, rmse, cc, area, extreme]
        results.append(result)
end_time = time.time() 

0






1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [3]:
print(end_time - start_time)
final = np.array(results)
print(final.shape)
np.savetxt('root_RNN.txt',final)
np.savetxt('root_RNN_pred.txt', np.array(predicted_vectors))

1929.5503180027008
(450, 5)


In [4]:
final

array([[1.00000000e-01, 5.61214423e-04, 9.54592429e-01, 7.33545648e-01,
        1.24955872e-03],
       [2.00000000e-01, 4.59909313e-04, 9.15556727e-01, 8.43949045e-01,
        1.02391119e-03],
       [3.00000000e-01, 3.34601466e-04, 9.31863535e-01, 9.00212314e-01,
        7.41741514e-04],
       ...,
       [7.00000000e-01, 2.86346305e-04, 9.11033305e-01, 8.87473461e-01,
        6.25114316e-04],
       [8.00000000e-01, 2.79055333e-04, 9.10653292e-01, 8.78980892e-01,
        6.06724015e-04],
       [9.00000000e-01, 3.28051433e-04, 8.76744397e-01, 7.23991507e-01,
        7.15787275e-04]])